# Importaciones
Primero tenemos que importar las librerias a usar.
- Numpy : libreria numerica de matrices.
- matplotlib: libreria para graficaion
- Pandas: libreria para manejar tablas
- Tabulate: extension de tablas

In [5]:
import pandas as pn
import numpy as np
import matplotlib as plt
import tabulate as tabl
from typing import Any

# Parte 1.
Para implementar el método simplex tenemos que introducir los "elementos principales":

<figure>
    <br>
    <center> <img src="./images/elementos.png"   style="width:500px;height:280px;" ></center>
</figure>

En este codigo dividiermos todo en 3 elementos:
1. $\LARGE a\_i$: Va a ser la matriz que contiene todos los coeficientes de las restricciones, de la forma $\large [A,I]$
2. $\LARGE b$ es el vector con los valores de las soluciones de las restricciones.
3. $\LARGE c$ es el vector con los coeficientes de las variables en la F.O.


In [6]:
a_i : np.ndarray[np.int_] = np.array([[-2,1,1,0,0],
                                      [-1,2,0,1,0],
                                      [1,0,0,0,1]])
b  : np.ndarray[np.int_] = np.array([2,7,3])
c  : np.ndarray[np.int_] = np.array([-1,-2,0,0,0])

# 2. Ahora debemos de implementar simplex.

In [20]:
# TODO: Actualmente el código lo mantengo con type: np.int_ para reducir errores. Una vez listo hay que cambiar el tipo para permitir float
def Simplex(A: np.ndarray[np.int_],b: np.ndarray[np.int_],c: np.ndarray[np.int_])-> Any:
    """
    Le pasamos los vectores con los datos del ejercicio. Utiliza un while loop para iterar hasta que se encuentre la solucion optima
    :param A: Matriz con coeficientes de las restricciones :type A: np.ndarrya[np.int_]
    :param b: Matriz con los valores de las soluciones de las restriciones :type  b: np.ndarrya[np.int_]: 
    :param c: Matriz con los coeficientes de la F.O. :type c: np.ndarrya[np.int_]
    :return: 
    """
    numero_restricciones = A.shape[0] # numero de condiciones
    numero_variables = A.shape[1]-numero_restricciones # numero de variables
    
    # Tenemos un problema ahora, las variables del array C esta escrito de la forma X_0, X_1, X_2, por lo tanto esta linea de codigo guardara los indices:
    indice_c : list[int] =[i for i in range(0,len(c))]
    
    # Coeficientes de las variables basicas
    coef_Vbasicas: np.ndarray[np.int_] = np.array(c[numero_variables:])
    # Coeficientes variables no básicas
    coef_VNbasicas: np.ndarray[np.int_] = np.array(c[:numero_restricciones])
    
    print(f"La cantidad de condiciones son: {numero_restricciones}")
    print(f"La cantidad de variables son: {numero_variables}\n")
    print(f"Los coeficientes de las variables basicas son: {coef_Vbasicas} \n")
    print(f"Los coeficientes de als variables NO basicas son: {coef_VNbasicas}\n")
    
    while True:
        # Mantenemos los indices de las variables basicas y no basicas aqui
        indice_actual_basica = indice_c[numero_variables:]
        indice_actual_noBasica = indice_c[:numero_restricciones]
        
        "Matriz Básica:" 
        B = A[:,   indice_actual_basica]
        B_inversa = np.linalg.inv(B)
        
        "Matriz de variables no básicas:"
        N= A[:,indice_actual_noBasica]
        
        "bHat son los valores de las variables basicas, con su resultao b"
        bHat=B_inversa @ b
        yT = coef_Vbasicas @ B_inversa
        
        "La siguiente linea de coidgo busca optimizar, determina cual variable va a entrar"
        cnHat: Any =  coef_VNbasicas - (yT @ N)
        
        #Encuentra el indice cn el valor minimo de cnHat, esta es la variable que va a entrar
        cnMinimo = np.argmin(cnHat)
        
        """La siguiente linea revisa si se encontró el mino, si es asi. Rompe el ciclo While True.
        Utiliza indice_actual_basica para obtener los indices de las variables+indices en bHat. 
        Los valores en Bhat son la solucion para cada una de las variables correspondientes.
        Si el valor es 0 es
        """
        if all(i >= 0 for i in cnHat):
            return coef_Vbasicas,  indice_actual_basica, coef_VNbasicas,indice_actual_noBasica,bHat,cnHat
        # Este es el indice para la columna de coeficientes dadas las variables
        
        indice = indice_c[cnMinimo]
        Ahat = B_inversa @ A[:,indice]
        
        # El siguiente código itera atravez de Ahata y bHat, y va a agarar los valores minimos siempre y cuando sean mayoeres a 0.
        # El valor mas pequeño mayor a cero, sera la variables que se transformara en no-básica
        
        ratios = []
        for _ in range(0,len(bHat)):
            Aval = Ahat[_]
            Bval = bHat[_]
            # Este filtro es para no ver a los ratio con valor menor o igual a cero, se anexara para mantener el indice.
            if Aval<= 0:
                ratios.append(10000000)
                continue
            ratios.append(Bval/Aval)
        # Se obtiene el valor minio:
        ratio_menor_indice = np.argmin(ratios)
        
        #Intercambiamos las variables basicas y no basicas usando los indices
        coef_VNbasicas[cnMinimo], coef_VNbasicas[ratio_menor_indice] = coef_Vbasicas[ratio_menor_indice], coef_VNbasicas[cnMinimo]
        # Intercambiamos el indice global, y el trackeador de indice:
        
    pass

In [21]:
Simplex(a_i,b,c)

La cantidad de condiciones son: 3
La cantidad de variables son: 2

Los coeficientes de las variables basicas son: [0 0 0] 

Los coeficientes de als variables NO basicas son: [-1 -2  0]


In [ ]:
Agolra